In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import difflib
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
!pip install scikit-learn

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
df=pd.read_csv("movies.csv")

In [6]:
df.head(10)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
5,5,258000000,Fantasy Action Adventure,http://www.sonypictures.com/movies/spider-man3/,559,dual identity amnesia sandstorm love of one's ...,en,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,115.699814,...,139.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,The battle within.,Spider-Man 3,5.9,3576,Tobey Maguire Kirsten Dunst James Franco Thoma...,"[{'name': 'Francine Maisler', 'gender': 1, 'de...",Sam Raimi
6,6,260000000,Animation Family,http://disney.go.com/disneypictures/tangled/,38757,hostage magic horse fairy tale musical,en,Tangled,When the kingdom's most wanted-and most charmi...,48.681969,...,100.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,They're taking adventure to new lengths.,Tangled,7.4,3330,Zachary Levi Mandy Moore Donna Murphy Ron Perl...,"[{'name': 'John Lasseter', 'gender': 2, 'depar...",Byron Howard
7,7,280000000,Action Adventure Science Fiction,http://marvel.com/movies/movie/193/avengers_ag...,99861,marvel comic sequel superhero based on comic b...,en,Avengers: Age of Ultron,When Tony Stark tries to jumpstart a dormant p...,134.279229,...,141.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Age Has Come.,Avengers: Age of Ultron,7.3,6767,Robert Downey Jr. Chris Hemsworth Mark Ruffalo...,"[{'name': 'Danny Elfman', 'gender': 2, 'depart...",Joss Whedon
8,8,250000000,Adventure Fantasy Family,http://harrypotter.warnerbros.com/harrypottera...,767,witch magic broom

In [7]:
df.shape

(4803, 24)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [9]:
df.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [10]:
req_data=df.iloc[:,[2,5,8,9,12,14,15,19,21,23]]

In [11]:
req_data

,genres,keywords,overview,popularity,release_date,runtime,spoken_languages,vote_average,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",7.2,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",6.9,Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",6.3,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,112.312950,2012-07-16,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",7.6,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca...",43.926995,2012-03-07,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",6.1,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...
4798,Action Crime Thriller,united states\u2013mexico barrier legs arms pa...,El Mariachi just wants to play his guitar and ...,14.269792,1992-09-04,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",6.6,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,Robert Rodriguez
4799,Comedy Romance,NaN,A newlywed couple's honeymoon is upended by th...,0.642552,2011-12-26,85.0,[],5.9,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,Edward Burns
4800,Comedy Drama Romance TV Movie,date love at first sight narration investigati...,"""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,2013-10-13,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",7.0,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,Scott Smith
4801,NaN,NaN,When ambitious New York attorney Sam is sent t...,0.857008,2012-05-03,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",5.7,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,Daniel Hsia


In [60]:
req_data.isnull().sum()

genres              0
keywords            0
overview            0
popularity          0
release_date        0
runtime             0
spoken_languages    0
vote_average        0
cast                0
director            0
dtype: int64

In [22]:
for i in req_data:
  req_data.loc[:,i]=req_data.loc[:,i].astype("string")

<ipython-input-22-571ac8fd9585>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data.loc[:,i]=req_data.loc[:,i].astype("string")
<ipython-input-22-571ac8fd9585>:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  req_data.loc[:,i]=req_data.loc[:,i].astype("string")


In [23]:
req_data.fillna("",inplace=True)

<ipython-input-23-8ae5a560f377>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_data.fillna("",inplace=True)


In [24]:
req_data.isnull().sum()

genres              0
keywords            0
overview            0
popularity          0
release_date        0
runtime             0
spoken_languages    0
vote_average        0
cast                0
director            0
dtype: int64

In [26]:
factors=req_data['keywords']+" "+req_data['director']+" "+req_data['overview']+" "+req_data['popularity']+" "+req_data['runtime']+" "+req_data ['spoken_languages']+" "+req_data['vote_average']
factors

0       culture clash future space war space colony so...
1       ocean drug abuse exotic island east india trad...
2       spy based on novel secret agent sequel mi6 Sam...
3       dc comics crime fighter terrorist secret ident...
4       based on novel mars medallion space travel pri...
                              ...                        
4798    united states\u2013mexico barrier legs arms pa...
4799     Edward Burns A newlywed couple's honeymoon is...
4800    date love at first sight narration investigati...
4801     Daniel Hsia When ambitious New York attorney ...
4802    obsession camcorder crush dream girl Brian Her...
Length: 4803, dtype: string

In [29]:
gen_vector=TfidfVectorizer().fit_transform(req_data["genres"])
rel_pop_vector=TfidfVectorizer().fit_transform(req_data["release_date"]+" "+req_data["popularity"])
cast_vector=TfidfVectorizer().fit_transform(req_data["cast"])
factors_vector=TfidfVectorizer().fit_transform(factors)

In [31]:
similar_genres=cosine_similarity(gen_vector)
similar_rel_pop=cosine_similarity(rel_pop_vector)
similar_cast=cosine_similarity(cast_vector)
similar_factors=cosine_similarity(factors_vector)

In [32]:
similar_genres

array([[1.        , 0.74495696, 0.42850395, ..., 0.        , 0.        ,
        0.        ],
       [0.74495696, 1.        , 0.57520631, ..., 0.        , 0.        ,
        0.        ],
       [0.42850395, 0.57520631, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [62]:
while (True):
    movie= input ("Enter Movie Name:")
    movie_name=difflib.get_close_matches(movie,df["title"])
    if (len(movie_name)==0):
      print("No recomendations found")
      print ("Try with other name")
    else:
      break
val=df[df.title==movie_name[0]]["index"].values[0]
movie=df.loc[[int (val)],["title"]].values[0][0]
genres=df.loc[[int (val)], ["genres"]].values[0][0]
cast=df.loc[[int (val)], ["cast"]].values[0][0]
release_date=df.loc[[int (val)], ["release_date"]].values[0][0]
print ("Movie Name:",movie)
print ("genres:",genres)
print(cast)
print(release_date)

Enter Movie Name:avater
Movie Name: Water
genres: Drama Foreign Romance
Lisa Ray Sarala John Abraham Seema Biswas Waheeda Rehman
2005-09-08


In [49]:
genre_score=list(enumerate(similar_genres[val]))
cast_score=list(enumerate(similar_cast[val]))
rel_pop_score=list(enumerate(similar_rel_pop[val]))
factors_score=list(enumerate(similar_factors[val]))

In [51]:
count=0
recommended_movies=[]
sorted_recomendations=sorted(genre_score,key=lambda x:x[1],reverse=True)
#print(sorted_recomendations)
for i in sorted_recomendations:
    index=i[0]
    recommended_movies.append(df[df. index==index] ['title'].values[0])
    count+=1
    if(count>(10)):
      break
#print(recommended_movies)
for i in recommended_movies:
  va=df[df.title==i]["homepage"].values [0]
  if pd.isnull(va):
    print(i)
  else:
    print(i,va)

[(3134, 1.0), (3226, 1.0), (3703, 1.0), (3732, 1.0), (3923, 1.0), (4371, 1.0), (2971, 0.9575782335869396), (3419, 0.9575782335869396), (4059, 0.9223899779625302), (2658, 0.9169823228870024), (3844, 0.9169823228870024), (4260, 0.9169823228870024), (4459, 0.9169823228870024), (4790, 0.9169823228870024), (2248, 0.8712621152136807), (3435, 0.8712621152136807), (3547, 0.8712621152136807), (4182, 0.8348564507496159), (3787, 0.8325466831994309), (4797, 0.8184024480276185), (3196, 0.811850053366506), (3729, 0.811850053366506), (3322, 0.7837088174603806), (3553, 0.7805810827637609), (4135, 0.7805810827637609), (3707, 0.7762595706292632), (3645, 0.7587145190946274), (3708, 0.7587145190946274), (4004, 0.7566542216205188), (2959, 0.7428957923237962), (4490, 0.685374466629394), (4666, 0.685374466629394), (4123, 0.6632867896531488), (3191, 0.6490878996480738), (49, 0.4751691473509692), (463, 0.4751691473509692), (493, 0.4751691473509692), (559, 0.4751691473509692), (648, 0.4751691473509692), (670, 0

In [52]:
count=0
recommended_movies=[]
sorted_recomendations=sorted(cast_score,key=lambda x:x[1],reverse=True)
#print(sorted_recomendations)
for i in sorted_recomendations:
    index=i[0]
    recommended_movies.append(df[df. index==index] ['title'].values[0])
    count+=1
    if(count>(10)):
      break
#print(recommended_movies)
for i in recommended_movies:
  va=df[df.title==i]["homepage"].values [0]
  if pd.isnull(va):
    print(i)
  else:
    print(i,va)

Water
All Hat http://allhatthemovie.com/
The Bridge of San Luis Rey http://www.thebridgeofsanluisreymovie.com/
Finding Forrester
Identity
The Grand Budapest Hotel
Last Action Hero
Amadeus
Lucky Numbers
Wild Hogs http://video.movies.go.com/wildhogs/
Beowulf http://www.beowulfmovie.com/


In [53]:
count=0
recommended_movies=[]
sorted_recomendations=sorted(rel_pop_score,key=lambda x:x[1],reverse=True)
#print(sorted_recomendations)
for i in sorted_recomendations:
    index=i[0]
    recommended_movies.append(df[df. index==index] ['title'].values[0])
    count+=1
    if(count>(10)):
      break
#print(recommended_movies)
for i in recommended_movies:
  va=df[df.title==i]["homepage"].values [0]
  if pd.isnull(va):
    print(i)
  else:
    print(i,va)

Water
The Man
An Unfinished Life http://www.miramax.com/anunfinishedlife/
Mrs Henderson Presents
The Exorcism of Emily Rose
Corpse Bride http://corpsebridemovie.warnerbros.com/
The Great Raid
The Chumscrubber
The Wendell Baker Story
Boynton Beach Club
Yes


In [54]:
count=0
recommended_movies=[]
sorted_recomendations=sorted(factors_score,key=lambda x:x[1],reverse=True)
#print(sorted_recomendations)
for i in sorted_recomendations:
    index=i[0]
    recommended_movies.append(df[df. index==index] ['title'].values[0])
    count+=1
    if(count>(10)):
      break
#print(recommended_movies)
for i in recommended_movies:
  va=df[df.title==i]["homepage"].values [0]
  if pd.isnull(va):
    print(i)
  else:
    print(i,va)

Water
Faith Connections
Earth
Gandhi, My Father http://gandhimyfather.erosentertainment.com/
Khiladi 786
Gandhi
Kites http://www.kites-thefilm.com/
Slumdog Millionaire http://www.foxsearchlight.com/slumdogmillionaire
Veer-Zaara
Housefull http://housefull.erosentertainment.com/
Rocket Singh: Salesman of the Year http://www.yashrajfilms.com/microsites/rocketsingh/rswebsite.html


In [63]:
import pickle

In [64]:
pickle.dump(df,open("movie_recm.pkl","wb"))

In [65]:
pickle.dump(similar_genres,open("similarity_genres.pkl","wb"))

In [66]:
pickle.dump(similar_rel_pop,open("similarity_rel_pop.pkl","wb"))

In [69]:
pickle.dump(similar_cast,open("similarity_cast.pkl","wb"))

In [70]:
pickle.dump(similar_factors,open("similarity_factors.pkl","wb"))